# Importações

In [ ]:
import os
import pandas as pd
import category_encoders as ce
from sklearn.preprocessing import StandardScaler

# Ambiente de Execução: Colab X VSCode

In [ ]:
# # Usar no Google Colab
# from google.colab import auth
# auth.authenticate_user()
# path_folder_bronze = '/content/'

In [ ]:
# Usar no VSCode
path_folder_silver = os.path.abspath('../data/temp/silver')

# Leitura dos Dados Limpos

In [ ]:
def extract_year_month(date): # Função para extrair o ano e o mês de uma data
    return date.strftime('%y%m')

In [ ]:
year_month = extract_year_month(pd.to_datetime('2017-06-01'))  # Exemplo de uso da função, como se fosse o dia 01 de junho de 2017

print(f'Year-Month: {year_month}')

In [ ]:
def read_csv_cleaned(path_folder_silver, file_name_silver): # Função para ler o arquivo CSV limpo
    df = pd.read_csv(os.path.join(path_folder_silver, file_name_silver))
    
    print(f'Arquivo "{file_name_silver}" lido.')
    return df

In [ ]:
file_name_silver = f'mta_{year_month}_cleaned.csv'

df = read_csv_cleaned(path_folder_silver, file_name_silver)

display(df.head(15))
display(df.info())

In [ ]:
def remove_columns(df, columns_to_remove): # Função para remover colunas do DataFrame
    df = df.drop(columns=columns_to_remove)
    print(f'Colunas removidas: {columns_to_remove}')
    return df

# Remoção de colunas não necessárias e de outliers

In [ ]:
df_ml = remove_columns(df, ['RecordedTimeRange', 'ScheduledTimeRange'])

display(df_ml.head(15))
display(df_ml.info())

In [ ]:
def remove_outliers_iqr(df, column_name): # Função para remover outliers usando o método IQR (Intervalo Interquartil)
    q1 = df[column_name].quantile(0.25)
    q3 = df[column_name].quantile(0.75)
    iqr = q3 - q1

    lower_bound = q1 - 1.5 * iqr
    upper_bound = q3 + 1.5 * iqr
    print(f'Limite inferior: {lower_bound}, Limite superior: {upper_bound}')

    df = df[(df[column_name] > lower_bound) & (df[column_name] < upper_bound)]
    
    df = df.reset_index(drop=True)

    print(f'Outliers removidos da coluna "{column_name}" usando IQR.')
    return df

In [ ]:
df_ml = remove_outliers_iqr(df_ml, 'DiffArrivalMins')

display(df_ml.head(15))
display(df_ml.info())

# Transformação de colunas de data e hora em numéricas para ML

In [ ]:
def transform_timedelta_to_seconds(df, column_name): # Função para transformar uma coluna de timedelta em segundos
    df[column_name] = pd.to_timedelta(df[column_name])
    df[column_name] = df[column_name].dt.total_seconds().astype(int)
    print(f'Coluna "{column_name}" transformada em segundos.')
    return df

In [ ]:
df_ml = transform_timedelta_to_seconds(df_ml, 'RecordedAtTime')
df_ml = transform_timedelta_to_seconds(df_ml, 'ScheduledArrivalTime')

display(df_ml.head(15))

In [ ]:
def transform_date_to_days(df, column_name): # Função para transformar uma coluna de date em dias desde uma data de referência
    df[column_name] = pd.to_datetime(df[column_name])
    
    stdate = pd.Timestamp('2017-01-01')  # Data de referência para o cálculo dos dias
    
    df['RecordedAtDate'] = (df['RecordedAtDate'] - stdate).dt.total_seconds()
    df['RecordedAtDate'] = df['RecordedAtDate'] // (24 * 3600)  # Convertendo para dias
    df['RecordedAtDate'] = df['RecordedAtDate'].astype(int)
    
    print(f'Coluna "{column_name}" transformada para dias desde a data de referência.')
    return df

In [ ]:
df_ml = transform_date_to_days(df_ml, 'RecordedAtDate')

display(df_ml.head(15))

# Transformação de colunas categoricas em numéricas com hash 

In [ ]:
def hash_encode_column(df, column_name): # Função para aplicar o HashingEncoder em uma coluna específica
    
    num_unique = df[column_name].nunique()
    print(f'Número de valores únicos na coluna "{column_name}": {num_unique}')
    
    # Determina o número de componentes para o HashingEncoder usando a maior potência de 2 menor ou igual ao número de valores únicos
    n_components = 1
    while True:
        if 2 ** n_components > num_unique:
            n_components += -1
            break
        n_components += 1
        
    n_components = 1 if n_components < 1 else n_components # Garantir que n_components seja pelo menos 1
    print(f'Número de componentes para HashingEncoder: {n_components}')
    
    encoder = ce.HashingEncoder(cols=[column_name], n_components=n_components)  # Ajuste n_components conforme necessário
    df = encoder.fit_transform(df)
    
    df = df.rename(columns={
        f'col_{i}': f'{column_name}_hash_{i}' 
        for i in range(n_components)
    })
    
    print(f'Coluna "{column_name}" codificada com HashingEncoder.')
    return df

In [ ]:
df_ml = hash_encode_column(df_ml, 'PublishedLineName')

display(df_ml.head(15))
display(df_ml.info())

In [ ]:
df_ml = hash_encode_column(df_ml, 'VehicleRef')

display(df_ml.head(15))
display(df_ml.info())

# Normalização das escalas das colunas de data e hora

In [ ]:
def standardize_columns(df, columns_names): # Função para padronizar colunas numéricas, ou seja, normalizar as escalas
    scaler = StandardScaler()
    df[columns_names] = scaler.fit_transform(df[columns_names])
    print(f'Colunas: "{columns_names}" padronizadas.')
    return df

In [ ]:
df_ml = standardize_columns(df_ml, ['RecordedAtTime', 'ScheduledArrivalTime', 'RecordedAtDate'])

display(df_ml.head(15))

# Salvar dados normalizados e pré-processados para ML

In [ ]:
def save_to_parquet(df, path_folder_gold_ml, file_name_gold_ml): # Função para salvar o DataFrame em formato Parquet
    df.to_parquet(os.path.join(path_folder_gold_ml, file_name_gold_ml), engine='pyarrow')

In [ ]:
path_folder_gold_ml = os.path.abspath(f'../data/temp/gold/ml/')
file_name_gold_ml = f'mta_{year_month}_ml.parquet'

save_to_parquet(df_ml, path_folder_gold_ml, file_name_gold_ml)